In [1]:
!pip install fastdtw

import numpy as np 
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw, dtw

In [2]:
%store -r dataset_series
%store -r digital_ink

In [3]:
dataset = np.array([dataset_series["x"], dataset_series["y"]])
dataset = np.swapaxes(dataset, 0, 1)

ink = np.array([digital_ink["x"], digital_ink["y"],])
ink = np.swapaxes(ink, 0, 1)

print(np.shape(dataset))
print(np.shape(ink))

(3952, 2)
(1335, 2)


In [4]:
distance, path = fastdtw(dataset, ink, dist=euclidean)
print(distance)

distance, path = dtw(dataset, ink, dist=euclidean)
print(distance)

1634.5065455509894
1019.1329267369531
